In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

In [ ]:
df_housing = pd.DataFrame(housing['data'])
df_housing.columns = housing['feature_names']
df_housing["median_house_value"] = housing['target']

In [ ]:
df_housing.head()

## Simple TF calculation from V1

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

In [ ]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

### To simplify above code

In [ ]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    result = f.eval()

print(result)

## TF V2

https://www.tensorflow.org/guide/migrate

V2 is eager execution, and instead sess.run, using function instead. Example below

**Note** TF will need a kernel restart

In [ ]:
import tensorflow as tf
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")

@tf.function
def calc(x, y):
    return x*x*y + y + 2

print(calc(x,y))

In [ ]:
W = tf.Variable(tf.ones(shape=(2,2)), name="W")
b = tf.Variable(tf.zeros(shape=(2)), name="b")

@tf.function
def forward(x):
  return W * x + b

out_a = forward([1,0])
print(out_a)

## Using TF to calculate Normal Equation

$
\hat{\theta} = (\boldsymbol{X}^T\cdot\boldsymbol{X})^-1\cdot\boldsymbol{X}^T\cdot\boldsymbol{y}
$

In [1]:
import numpy as np
import tensorflow as tf

from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

# getting shapes of the input dataset
print("X shape {}, y shape {}".format(housing['data'].shape, housing['target'].shape))
m, n = housing['data'].shape

X shape (20640, 8), y shape (20640,)


In [2]:
housing_data_plus_bias = np.c_[np.ones(m), housing['data']]

In [3]:
X = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing['target'].reshape(-1, 1), dtype = tf.float32, name = "y")

In [4]:
@tf.function
def normal_equation(X, y):
    XT  = tf.transpose(X)
    theta = tf.matmul(tf.matmul(tf.linalg.inv(tf.matmul(XT, X)), XT), y)
    return theta


theta = normal_equation(X, y)

## Using implement gradient descent

- One way is to calculate

In [10]:
@tf.function
def gradient_descent(X, y, n_epochs = 100, learning_rate = 0.01):
    m, n = X.shape
    theta = tf.Variable(initial_value = tf.random.uniform([n, 1], -1.0, 1.0), name = "theta")  
    
    for epoch in range(n_epochs):                
        with tf.GradientTape(persistent=True) as tape:
          y_pred = X @ theta
          error = y_pred - y
          loss = tf.reduce_mean(error ** 2)
    
        # gradient is calculated for every variable
        grad = tape.gradient(loss,[X, y, theta])        
        theta.assign(theta - learning_rate * grad[2])
        
        if (epoch % 10000 == 0):
         print(loss)
                    
    return theta

tf.config.run_functions_eagerly(True)
theta = gradient_descent(X, y, 50000, 0.0000001)

tf.Tensor(161043.1, shape=(), dtype=float32)
tf.Tensor(28.282642, shape=(), dtype=float32)
tf.Tensor(19.239275, shape=(), dtype=float32)
tf.Tensor(13.394658, shape=(), dtype=float32)
tf.Tensor(9.6084795, shape=(), dtype=float32)


In [11]:
theta

<tf.Variable 'theta:0' shape=(9, 1) dtype=float32, numpy=
array([[-3.64643663e-01],
       [ 3.95412333e-02],
       [ 7.41983578e-02],
       [-1.57197013e-01],
       [-9.46455538e-01],
       [ 3.93365626e-04],
       [-1.93999767e-01],
       [ 4.42019105e-01],
       [ 1.14545025e-01]], dtype=float32)>

### How GradientTape works

In [20]:
x = tf.Variable(.3)
with tf.GradientTape() as tape:
    y = x ** 2

dy_dx = tape.gradient(y, x)
dy_dx.numpy()

0.6